In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)


## importing data

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

from datetime import datetime
from datetime import date

## Features 
* Age
* sex
* income_per_annum
* occupation
* Z20
* app_status
* E2 channel
* iskarvy
* KRA
<br>
<br>
* number of logins in last year
* num_trades in last year
* months since last login
* months since last trade
* FAO segment trading client
* delivery segment trading client
* intraday segment trading client
* dp_holding
* ledger_balance


In [7]:
df_model.columns

Index(['Unnamed: 0', 'client_code', 'KRA YN', 'IsKarvy', 'cm_sex', 'IncomePA',
       'IsDormant', 'E2Channel', 'target', 'age', 'months_since_last_login',
       'months_since_last_futures_trade', 'months_since_last_options_trade',
       'months_since_last_intraday_trade', 'months_since_last_delivery_trade',
       'THV', 'ALB', 'Z20', 'total_logins_one_year', 'total_trades_one_year'],
      dtype='object')

## Data importing

In [8]:
df_client = pd.read_csv("../../../data/dw_mst_client.csv")
# df_app_clients = pd.read_csv("../../../data/app_clients.csv")

# df_FAO = pd.read_csv("../../../data/FAO_clients.csv")
df_ledger = pd.read_csv("../../../data/THV_ALB_all_clients_20211114.csv")
df_etf = pd.read_csv("../../../data/etf_accepted_till_20211115.csv")

df_z20 = pd.read_csv("../../../data/z20_20211115.csv")


df_client =  df_client[df_client["data_source"] == "Equity"]


FileNotFoundError: [Errno 2] No such file or directory: '../../../data/dw_mst_client.csv'

In [ ]:
df_model = df_client[["cm_cd", "cm_dob", "KRA YN", "IsKarvy", "cm_sex", "Last_MobileLoginDate", "IncomePA", 
                      "IsDormant", "LTD", "Futures LTD", "Options LTD", "IntraDay LTD", "Delivery LTD", "E2Channel"]]

########################### adding target feature ##################################
df_model = pd.merge(df_model, 
                   df_etf, left_on = "cm_cd", 
                   right_on = "ClientCode", how = "right")[["ClientCode", "SegActDatetime"] + list(df_model.columns)[1:]]

df_model["SegActDatetime"] = df_model["SegActDatetime"].astype(str).str.split(" ").str[0].str.split("-").str.join("").astype(int)


## adding target feature
df_model["target"] = df_model["LTD"] - df_model["SegActDatetime"]
# df_model.loc[df_model["target"] < 0, 'target'] = 1
# df_model.loc[df_model["target"] > 0, 'target'] = 0


In [ ]:
df_model

In [ ]:
(df_model["target"] > 0).sum(), (df_model["target"] < 0).sum()

## addding Features

In [ ]:
### short listing clients
now = datetime.now()

# taking only clients who opened ther account after 2010
df_client = df_client[(df_client["cm_opendt"] >= 20101001) & (df_client["data_source"] == "Equity")]
df_model = df_client[["cm_cd", "cm_dob", "KRA YN", "IsKarvy", "cm_sex", "Last_MobileLoginDate", "IncomePA", 
                      "IsDormant", "LTD", "Futures LTD", "Options LTD", "IntraDay LTD", "Delivery LTD", "E2Channel"]]

########################### adding target feature ##################################
df_model = pd.merge(df_model, 
                   df_etf, left_on = "cm_cd", 
                   right_on = "ClientCode", how = "right")[["ClientCode", "ETFDatetime"] + list(df_model.columns)[1:]]

df_model["ETFDatetime"] = df_model["ETFDatetime"].astype(str).str.split(" ").str[0].str.split("-").str.join("").astype(int)

df_model = df_model[df_model["ETFDatetime"] > 20000101]
## adding target feature
df_model["target"] = df_model["LTD"] - df_model["ETFDatetime"]
df_model['target'][df_model["target"] > 0] = 1
df_model['target'][df_model["target"] < 0] = 0
del df_model["ETFDatetime"]
del df_model["LTD"]

####################################################################################




## adding age 
df_model["cm_dob"] = df_model["cm_dob"].astype(str).str.split(".").str[0]
df_model["cm_dob"] = pd.to_datetime(df_model["cm_dob"], errors = "coerce")
df_model["age"] =  (now -df_model["cm_dob"]).astype('<m8[Y]')
del df_model["cm_dob"]

print("age column added")
print("-------------------------------------------------")
# adding months since last trade
df_model["Last_MobileLoginDate"] = df_model["Last_MobileLoginDate"].astype(str).str.split(".").str[0]
df_model["Last_MobileLoginDate"] = pd.to_datetime(df_model["Last_MobileLoginDate"], errors = "coerce")
df_model["months_since_last_login"] =  (now -df_model["Last_MobileLoginDate"])/np.timedelta64(1, 'M')
del df_model["Last_MobileLoginDate"]
df_model["months_since_last_login"] = df_model["months_since_last_login"].fillna(132)

print("months since last login added")
print("-------------------------------------------------")

# adding months since last trade - futures
df_model["Futures LTD"] = df_model["Futures LTD"].astype(str).str.split(".").str[0]
df_model["Futures LTD"] = pd.to_datetime(df_model["Futures LTD"], errors = "coerce")
df_model["months_since_last_futures_trade"] =  (now -df_model["Futures LTD"])/np.timedelta64(1, 'M')
del df_model["Futures LTD"]
df_model["months_since_last_futures_trade"] = df_model["months_since_last_futures_trade"].fillna(132)


print("months since last furures trade added")
print("-------------------------------------------------")

# adding months since last trade - options
df_model["Options LTD"] = df_model["Options LTD"].astype(str).str.split(".").str[0]
df_model["Options LTD"] = pd.to_datetime(df_model["Options LTD"], errors = "coerce")
df_model["months_since_last_options_trade"] =  (now -df_model["Options LTD"])/np.timedelta64(1, 'M')
del df_model["Options LTD"]
df_model["months_since_last_options_trade"] = df_model["months_since_last_options_trade"].fillna(132)
""

print("months since last options trade added")
print("-------------------------------------------------")

# adding months since last trade - intraday
df_model["IntraDay LTD"] = df_model["IntraDay LTD"].astype(str).str.split(".").str[0]
df_model["IntraDay LTD"] = pd.to_datetime(df_model["IntraDay LTD"], errors = "coerce")
df_model["months_since_last_intraday_trade"] =  (now -df_model["IntraDay LTD"])/np.timedelta64(1, 'M')
del df_model["IntraDay LTD"]
df_model["months_since_last_intraday_trade"] = df_model["months_since_last_intraday_trade"].fillna(132)

print("months since last intraday trade added")
print("-------------------------------------------------")

# adding months since last trade - delivery
df_model["Delivery LTD"] = df_model["Delivery LTD"].astype(str).str.split(".").str[0]
df_model["Delivery LTD"] = pd.to_datetime(df_model["Delivery LTD"], errors = "coerce")
df_model["months_since_last_delivery_trade"] =  (now -df_model["Delivery LTD"])/np.timedelta64(1, 'M')
del df_model["Delivery LTD"]
df_model["months_since_last_delivery_trade"] = df_model["months_since_last_delivery_trade"].fillna(132)


print("months since last del trade added")
print("-------------------------------------------------")




In [ ]:
## adding THV and ALB
df_model = pd.merge(df_model, df_ledger, 
                   left_on = "ClientCode", 
                   right_on = "LoginID", how = "left")[list(df_model.columns) + ["THV", "ALB"]]

In [ ]:
## adding z20 
df_z20["Z20"] = [1]*len(df_z20)
df_model = pd.merge(df_model, df_z20, 
                   left_on = "ClientCode", 
                   right_on = "cm_cd", how = "left")[list(df_model.columns) + ["Z20"]]
df_model["Z20"] = df_model["Z20"].fillna(0)

In [ ]:
df_model = df_model[~df_model['ClientCode'].duplicated(keep='last')]

In [ ]:
# ## adding logins and trades in one year
# df_logins_one_year = pd.read_csv("../../../data/logins_in_one_year.csv") 
# df_model.rename(columns = {"ClientCode": "client_code"}, inplace = True)
# df_model = pd.merge(df_model, df_logins_one_year, 
#                    left_on = "client_code", 
#                    right_on = "ClientCode", how = "left")[list(df_model.columns) + ["total_logins_one_year"]]
# df_model["total_logins_one_year"] = df_model["total_logins_one_year"].fillna(0)

df_trades_one_year = pd.read_csv("../../../data/trades_in_one_year.csv") 
df_model = pd.merge(df_model, df_trades_one_year, 
                   left_on = "client_code", 
                   right_on = "Client_Code", how = "left")[list(df_model.columns) + ["total_trades_one_year"]]
df_model["total_trades_one_year"] = df_model["total_trades_one_year"].fillna(0)

In [ ]:
df_model

In [ ]:
df_model.to_csv("../inter_data/preprocessed.csv")

## Analysis

In [5]:
df_model = pd.read_csv("../inter_data/preprocessed.csv")

In [ ]:
df_model["target"].value_counts()

In [ ]:
sns.set(rc={'figure.figsize':(20,5)})
df_model["IncomePA"] = df_model["IncomePA"].replace({"5_10L ??": "5_10L", 
                                        "<=1??": "<=1L", 
                                        "1_5??": "1_5L", "<= 1L": "<=1L"})
sns.countplot("IncomePA",data = df_model, hue = "target").set_title("income wise analysis",  fontsize=25)


In [ ]:
sns.set(rc={'figure.figsize':(20,5)})

sns.countplot("E2Channel",data = df_model, hue = "target", 
              order = df_model["E2Channel"].value_counts().index[:6], palette = "rocket").set_title("channel wise analysis",  fontsize=25)


In [ ]:
sns.set(rc={'figure.figsize':(20, 8)})

sns.histplot(data = df_model, y = "age", hue = "target", 
              bins = 10, multiple = "stack").set_title("channel wise analysis",  fontsize=20)


In [ ]:
fig, axs = plt.subplots(ncols=2)

sns.distplot(df_model[df_model["total_logins_one_year"] < 5000][df_model["target"] == 0]["total_logins_one_year"], ax=axs[0]).set_title("Distribution plot of non trading logins in last one year",  fontsize=20)
sns.distplot(df_model[df_model["total_logins_one_year"] < 5000][df_model["target"] == 1]["total_logins_one_year"], ax=axs[1]).set_title("Distribution plot of trading clients logins in last one year ",  fontsize=20)

In [ ]:
fig, axs = plt.subplots(ncols=2)

sns.distplot(df_model[df_model["total_trades_one_year"] < 1000][df_model["target"] == 0]["total_trades_one_year"], 
             ax=axs[0]).set_title("Distribution plot of non-trading clients trades in last one year",  fontsize=20)
sns.distplot(df_model[df_model["total_trades_one_year"] < 1000][df_model["target"] == 1]["total_trades_one_year"], 
             ax=axs[1]).set_title("Distribution plot of trading clients trades in last one year ",  fontsize=20)

In [ ]:
fig, axs = plt.subplots(ncols=2)

sns.distplot(df_model[(df_model["target"] == 0) & (df_model["months_since_last_delivery_trade"] < 20)]["months_since_last_delivery_trade"], 
            ax=axs[0]).set_title("Distribution plot of non trading clients months since last trade in last one year ",  fontsize=15)
sns.distplot(df_model[(df_model["target"] == 1) & (df_model["months_since_last_delivery_trade"] < 20)]["months_since_last_delivery_trade"], 
            ax=axs[1]).set_title("Distribution plot of trading clients months since last trade in last one year ",  fontsize=15)

In [ ]:
print("mean of number of trades in one year by clients trading post etf: {}".format(df_model[df_model["target"] == 1]["total_trades_one_year"].median()))
print("mean of number of trades in one year by clients not trading post etf: {}".format(df_model[df_model["target"] == 0]["total_trades_one_year"].mean()))

In [6]:
df_model

,Unnamed: 0,client_code,KRA YN,IsKarvy,cm_sex,IncomePA,IsDormant,E2Channel,target,age,months_since_last_login,months_since_last_futures_trade,months_since_last_options_trade,months_since_last_intraday_trade,months_since_last_delivery_trade,THV,ALB,Z20,total_logins_one_year,total_trades_one_year
0,0,RAPT1001,Y,0.0,M,5_10L,N,Franchisee,0.0,34.0,132.000000,132.0,132.000000,132.000000,3.704233,0.0,0.18,1.0,0.0,1.0
1,1,SIQBAL22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,132.000000,132.0,132.000000,132.000000,132.000000,39.0,5.82,0.0,47.0,1.0
2,2,OPA10002,Y,0.0,M,5_10L,N,Franchisee,1.0,25.0,0.221616,132.0,132.000000,2.981426,3.211410,0.0,5.77,1.0,266.0,36.0
3,3,52093616,Y,0.0,M,1_5L,N,CAT,1.0,46.0,0.155906,132.0,132.000000,0.911568,0.155906,27634.0,7214.16,1.0,493.0,131.0
4,4,SIG10103,Y,0.0,M,5_10L,N,Franchisee,1.0,36.0,0.287325,132.0,1.535811,132.000000,2.094344,14693.0,0.15,1.0,282.0,52.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359242,359242,PIDESHNA,Y,0.0,M,5_10L,N,Franchisee,NaN,36.0,132.000000,132.0,132.000000,132.000000,132.000000,0.0,1.46,1.0,0.0,1.0
359243,359243,PRMA3411,N,0.0,M,5_10L,N,Premia,NaN,65.0,132.000000,132.0,132.000000,132.000000,132.000000,0.0,1.38,1.0,0.0,1.0
359244,359244,PUND2371,Y,0.0,M,1_5L,N,CAT,NaN,26.0,0.155906,132.0,132.000000,132.000000,132.000000,0.0,101.39,1.0,20.0,1.0
359245,359245,MMSM0103,N,0.0,M,5_10L,N,Franchisee,0.0,23.0,0.155906,132.0,132.000000,0.583019,0.418745,36600.0,3845.42,1.0,227.0,18.0
